<a href="https://colab.research.google.com/github/raghavkaushik/DeepLearning/blob/main/CNN_ResNetv1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/PacktPublishing/Advanced-Deep-Learning-with-Keras/blob/master/chapter2-deep-networks/resnet-cifar10-2.2.1.py

This following program is a practice program based upon the ResNetV1 from the above mentioned Gibtub code. 

In [1]:
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, BatchNormalization, GlobalMaxPooling2D, AveragePooling2D, Input, Activation, add
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import math
import os

In [2]:
(Xtrain,Ytrain),(Xtest,Ytest)=cifar10.load_data()
Xtrain,Xtest=Xtrain/255.0,Xtest/255.0

In [3]:
#We are using the ResNet20 for the following classification.
#For version 1 the depth of the model is based upon - 
n=3
version=1
depth=n*6+2
subtract_pixel_mean=True
num_classes=10
data_augmentation = True
modeltype='ResNet%dv%d'%(depth,version)
print("The ResNet depth and it's version:",modeltype)

The ResNet depth and it's version: ResNet20v1


In [4]:
input_shape = Xtrain.shape[1:]
print("The shape of input:", input_shape)

The shape of input: (32, 32, 3)


In [5]:
#if we subtract pixel mean(It is a process where one pixel or another image is subtracted from the image to level the image or detect changes between them):
if subtract_pixel_mean:
  xtrain_mean=np.mean(Xtrain,axis=0)
  Xtrain-=xtrain_mean
  Xtest-=xtrain_mean

In [6]:
print('Xtrain shape:',Xtrain.shape)
print('Train samples:',Xtrain.shape[0])
print('Test samples:',Xtest.shape[0])
print('Ytrain shape:',Ytrain.shape)

Xtrain shape: (50000, 32, 32, 3)
Train samples: 50000
Test samples: 10000
Ytrain shape: (50000, 1)


In [7]:
#Convert class vectors to binary class matrics:
Ytrain=to_categorical(Ytrain,num_classes)
Ytest=to_categorical(Ytest,num_classes)

In [8]:
#The scheduler is called automatically to reduce the learning rate after a certain number of epochs which are defined within the program. input: Epochs and Output: Lr
#0.001 = 1e-3(number mentions the position of 1 after zeros)
def lr_schedule(epoch):
  lr=1e-3
  if epoch>180:
    lr*=0.5e-3
  elif epoch>160:
    lr*=1e-3
  elif epoch>120:
    lr*=1e-2
  elif epoch>80:
    lr*=1e-1
  print(f"Learning rate at{epoch} is {lr}")
  return lr


In [9]:
#defining the resnet_layer function which will be used/called in residual blocks to perform convolution.
def resnet_layer(inputs,num_filters=16,kernel_size=3, strides=1,activation='relu', batch_normalization=True, conv_first=True):
  conv=Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))
  x=inputs
  if conv_first:
    x=conv(x)
    if batch_normalization:
      x=BatchNormalization()(x)
    if activation is not None:
      x=Activation(activation)(x)
  else:
    if batch_normalization:
      x=BatchNormalization()(x)
    if activation is not None:
      x=Activation(activation)(x)
    x=conv(x)
  return x

In [10]:
def resnetv1(input_shape,depth,num_classes=10):
  if (depth-2)%6!=0:
    raise ValueError("Depth should be 6n+2(eg:20,32...")
  # start model definition.
  num_filters=16
  num_res_blocks=int((depth-2)/6)
  inputs=Input(shape=input_shape)
  x=resnet_layer(inputs=inputs)
  for stack in range(3):
    for res_block in range(num_res_blocks):
      strides=1
      if stack>0 and res_block==0:
        strides=2
      y=resnet_layer(inputs=x,
                       num_filters=num_filters,
                       strides=strides)
      y=resnet_layer(inputs=y,
                       num_filters=num_filters,
                       activation=None)
      if stack > 0 and res_block == 0:
        x=resnet_layer(inputs=x,
                       num_filters=num_filters,
                       kernel_size=1,
                       strides=strides,
                       activation=None,
                       batch_normalization=False)
      x = add([x, y])
      x=Activation('relu')(x)
    num_filters*=2
  x=AveragePooling2D(pool_size=8)(x)
  y=GlobalMaxPooling2D()(x)
  outputs=Dense(num_classes, activation='softmax', kernel_initializer='he_normal')(y)
  model=Model(inputs=inputs,outputs=outputs)
  return model

In [11]:
model=resnetv1(input_shape=input_shape, depth=depth)

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=lr_schedule(0)),
              metrics=['acc'])

model.summary()

Learning rate at0 is 0.001
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 16)   448         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 16)  64          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 32, 32, 16)   0           ['

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [15]:
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar10_%s_model.{epoch:03d}.h5' 
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler=LearningRateScheduler(lr_schedule)
lr_reducer=ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)
callbacks = [checkpoint,lr_reducer, lr_scheduler]

In [ ]:
if not data_augmentation:
  model.fit(Xtrain,Ytrain, batch_size=32, epochs=200, validation_data=(Xtest,Ytest), shuffle=True, callbacks=callbacks)
else:
  print("Using real-time data-augmentation")
  datagen=ImageDataGenerator(# set input mean to 0 over the dataset
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=0,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False)
  datagen.fit(Xtrain)
  steps_per_epochs=math.ceil(len(Xtrain)/32)
  model.fit(x=datagen.flow(Xtrain, Ytrain, batch_size=32),
              verbose=1,
              epochs=200,
              validation_data=(Xtest, Ytest),
              steps_per_epoch=steps_per_epochs,
              callbacks=callbacks)

Using real-time data-augmentation
Learning rate at0 is 0.001
Epoch 1/200
1563/1563 [==============================] - ETA: 0s - loss: 1.5816 - acc: 0.4822
Epoch 00001: val_acc improved from -inf to 0.52330, saving model to /content/saved_models/cifar10_%s_model.001.h5


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


1563/1563 [==============================] - 724s 462ms/step - loss: 1.5816 - acc: 0.4822 - val_loss: 1.5252 - val_acc: 0.5233 - lr: 0.0010
Learning rate at1 is 0.001
Epoch 2/200
 759/1563 [=============>................] - ETA: 6:07 - loss: 1.2546 - acc: 0.6084

In [ ]:
scores = model.evaluate(x_test,
                        y_test,
                        batch_size=batch_size,
                        verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])